## Handling of results from a single run of the simulation

In [43]:
# Import libraries
import pandas as pd
import json
import os 
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

In [1]:
# The settings of the current experiment
model = "3.5"
temperature = 1
initial_examples_generated = 10
feedback_iterations = 10
reboot = 100000
specifications = "all"
output_folder = f"{model}-{initial_examples_generated}-{feedback_iterations}-{temperature}-{specifications}"

# Print the command to generate the code
command = f"python3 main.py generate_code_folder -d ../no_loop_problems/ -ieg {initial_examples_generated} -iter {feedback_iterations} -temp {temperature}  -reboot {reboot} -wpt 5 -o ../output/{output_folder} -output-file generated_code.c"

print(command)


SyntaxError: invalid syntax (267147248.py, line 6)

In [67]:
# print all folders in the directory sorted by name
folders = os.listdir('../no_loop_problems/')

# Sort on number
folders.sort(key=lambda x: int(x.split('-')[0]))

# Create an empty dataframe that will be filled with info of the iterations
iteration_array = []

for folder_name in folders:
    # Read the output given in the file 
    with open(f"../output/{output_folder}/{folder_name}/results.txt", 'r') as file:
        # Read the file which contains an array
        data = json.load(file)
        
    verified_goals = []
    passed_tests = []

    # for each iteration in the array add it to the dataframe
    for i in data:
        # print the amount of passed tests
        verified_goals.append(i['verified_goals'])
        passed_tests.append(f"{i['test_information'][-1]['summary']['passed']} / {i['test_information'][-1]['summary']['total']}")

    # Add it as a column to the dataframe
    iteration_array.append(verified_goals)
    iteration_array.append(passed_tests)

# Create a dataframe with the information of the iterations
df = pd.DataFrame(iteration_array)

## Work together with google sheets and google drive to store the information and results


In [68]:
# define the scope
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']

# add credentials to the account
creds = ServiceAccountCredentials.from_json_keyfile_name('../tmp/vecogen-1ec545f23e03.json', scope)

# authorize the clientsheet 
client = gspread.authorize(creds)

# get the instance of the Spreadsheet
sheet = client.open('Master Thesis Merlijn Results')

# get the first sheet of the Spreadsheet
template_sheet_name = "template-model-initial_examples_generated-feedback_iterations-temperature-dataset"
template_sheet = sheet.worksheet(template_sheet_name)

# Copy the sheet called "template" to a new sheet
template_sheet_name = "template-model-initial_examples_generated-feedback_iterations-temperature-dataset"
new_sheet_name = f"{model}-{initial_examples_generated}-{feedback_iterations}-{temperature}-{specifications}"

# Copy the template sheet to a new sheet
result_sheet = sheet.duplicate_sheet(template_sheet.id)
result_sheet.update_title(new_sheet_name)

# Change the order to the last sheet
sheet.batch_update({
    "requests": {
        "updateSheetProperties": {
            "properties": {
                "sheetId": result_sheet.id,
                "index": len(sheet.worksheets()) - 1  # Move to the last position
            },
            "fields": "index"
        }
    }
})

# Add the data to the sheet
worksheet = sheet.worksheet(new_sheet_name)

# Add the data to the sheet
start_cell = 'E10'
end_cell =  'ZZZ69'

# Convert DataFrame to list of lists
data_to_update = df.values.tolist()

# Update the range with the DataFrame values
worksheet.update(start_cell + ':' + end_cell, data_to_update)

# Put in cell D1 the model used
worksheet.update_acell('D1', model)
worksheet.update_acell('D2', temperature)
worksheet.update_acell('D3', initial_examples_generated)
worksheet.update_acell('D4', feedback_iterations)
worksheet.update_acell('D5', specifications)
worksheet.update_acell('D6', command)

C:\Users\MAL Sevenhuijsen\AppData\Local\Temp\ipykernel_5576\3705707450.py:49: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(start_cell + ':' + end_cell, data_to_update)


{'spreadsheetId': '1QgsGGnlo6DWp0rS5wwbnouUjDsMZSUqd3fofFc2aKAQ',
 'updatedRange': "'3.5-10-10-1-all'!D6",
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

## Store the data in the google drive folder    

In [ ]:
# Define the scope and authenticate using service account credentials
SCOPES = ['https://www.googleapis.com/auth/drive']
creds = Credentials.from_service_account_file('../tmp/vecogen-1ec545f23e03.json', scopes=SCOPES)

# Authenticate Google Drive API client
drive_service = build('drive', 'v3', credentials=creds)

def create_folder(name, parent_id=None):
    """Create a folder in Google Drive."""
    folder_metadata = {
        'name': name,
        'mimeType': 'application/vnd.google-apps.folder'
    }
    if parent_id:
        folder_metadata['parents'] = [parent_id]
    folder = drive_service.files().create(body=folder_metadata, fields='id').execute()
    return folder.get('id')

def upload_file(file_path, parent_id):
    """Upload a file to Google Drive."""
    file_name = os.path.basename(file_path)
    media = MediaFileUpload(file_path, resumable=True)
    file_metadata = {
        'name': file_name,
        'parents': [parent_id]
    }
    file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    return file.get('id')

def upload_folder(local_folder_path, parent_id=None):
    """Upload a folder to Google Drive, recursively including all files and subfolders."""
    folder_name = os.path.basename(local_folder_path)
    folder_id = create_folder(folder_name, parent_id)
    for item in os.listdir(local_folder_path):
        item_path = os.path.join(local_folder_path, item)
        if os.path.isdir(item_path):
            upload_folder(item_path, folder_id)
        else:
            upload_file(item_path, folder_id)

def list_folders():
    """List all folders in Google Drive."""
    query = "mimeType='application/vnd.google-apps.folder' and trashed=false"
    results = drive_service.files().list(q=query, spaces='drive', fields='nextPageToken, files(id, name)').execute()
    folders = results.get('files', [])

    if not folders:
        print('No folders found.')
    else:
        print('Folders:')
        for folder in folders:
            print(f"Name: {folder['name']}, ID: {folder['id']}")

# Define the local folder you want to upload and the Google Drive parent folder ID
local_folder_path = f'../output/{output_folder}'
drive_parent_folder_id = '1ZvsBlLV94f1TTG-2aIfeX_v86I4bOxq8'

# Start the folder upload process
upload_folder(local_folder_path, drive_parent_folder_id)

print(f'Folder {local_folder_path} uploaded to Google Drive folder with ID {drive_parent_folder_id}')


Folder ../output/3.5-100-0-1-all-all uploaded to Google Drive folder with ID 1ZvsBlLV94f1TTG-2aIfeX_v86I4bOxq8
